# Demo Notebook

In [1]:
# your other imports here ...
import sys, os
import pandas as pd

# TODO: replace with your path/to/ninarow
ninarowdir = os.path.dirname(os.getcwd())
modelfitdir = ninarowdir + "/model_fitting/"
# os.listdir(modelfitdir)

# sets the import path to the model-fitting directory
sys.path.insert(0, modelfitdir)
from parsers import *
from model_fit import *
import model_fit

# WARNING: DO NOT USE %load_ext autoreload and %autoreload 2 as it interferes with 
# the Multi-threading processes!

## Data Handling

### File Formatting
The data columns should be ordered: 

    - black_pieces (binary), 
    - white_pieces (binary), 
    - player_color (Black/White), 
    - move (binary), 
    - response time (not used in fitting), 
    - [group_id] (optional), 
    - participant_id

for more info, see `parsers.py`

In [2]:
data_path = "../data"
df = pd.read_pickle(f"{data_path}/data.pkl")
df["response_time"] = 1
df["participant_id"] = 1 # in the demo, there is only one participant, but if you have multiple, you may want to change this
df["black"] = df["bp"]
df["white"] = df["wp"]
df = df[:10][["black", "white", "color", "move", "response_time", "participant_id"]]
df.to_csv(f"{data_path}/data.csv", index = False)

In [3]:
# TODO: make a folder (here I've called it "data/")
# which holds your data in a csv called data.csv and put its directory here ...
data_dir = "../data"
data_csv = f"{data_dir}/data.csv"
df = pd.read_csv(data_csv)

This is what our data looks like in our CSV ...

## Cross Validation Splits

In [4]:
from utils import make_splits

In [5]:
splits = make_splits(df, output_dir = data_dir)

Saving split1 to ../data/1.csv
Saving split2 to ../data/2.csv
Saving split3 to ../data/3.csv
Saving split4 to ../data/4.csv
Saving split5 to ../data/5.csv


In [6]:
splits[0].head()

,black,white,color,move,response_time,participant_id
0,8192,0,White,4194304,1,1
1,2684354560,4194304,White,2097152,1,1


## Load Data into Model Fitting Code

In [7]:
from utils import *
[csvmove for csvmove in df_to_CSVMove(splits[0])][0]

8192	0	White	4194304	1.0	1	1

The parser takes in a CSV filename and turns it into a list of 
objects of type CSVMove ...

In [8]:
data = parse_participant_file(f"{data_path}/1.csv", ignore_csv_header = True)
print(f"the rows are type {type(data[0])}")
print("and they look like")
print(data[:10])

File is either not a JSON file, or is malformed. Attempting to parse as a CSV...
the rows are type <class 'parsers.CSVMove'>
and they look like
[8192	0	White	4194304	1.0	1	1, 2684354560	4194304	White	2097152	1.0	1	1]


### Making Cross Validation Splits 

In [9]:
data_path = "../data"
output_path = "../data/out"
n_splits = 5
fold_number = 1
threads = 1
random_sample = False
verbose = True


assert np.all([f"{i + 1}.csv" in os.listdir(data_path) for i in range(n_splits)])

print("Detected splits in this directory. Loading splits ...")
splits = [pd.read_csv(f"{data_dir}/{i + 1}.csv") for i in range(n_splits)]
fold_data = [[csvmove for csvmove in df_to_CSVMove(split, warn = False)] for split in splits]

print(f"Building output directory at {output_path}")
os.makedirs(output_path, exist_ok = True)

Detected splits in this directory. Loading splits ...
Building output directory at ../data/out


In [11]:
from multiprocessing import Pool, Value, set_start_method

random.seed(10)
initialize_global_threads(1)
model_fitter = ModelFitter(DefaultModel(), 
                           random_sample = random_sample, 
                           verbose = verbose, 
                           threads = threads)

params, loglik_train, loglik_test = model_fitter.cross_validate(fold_data, fold_number - 1)
# with (output_path / ("params" + str(i + 1) + ".csv")).open('w') as f:
#     f.write(','.join(str(x) for x in params))
# with (output_path / ("lltrain" + str(i + 1) + ".csv")).open('w') as f:
#     f.write(','.join(str(x) for x in loglik_train))
# with (output_path / ("lltest" + str(i + 1) + ".csv")).open('w') as f:
#     f.write(' '.join(str(x) for x in loglik_test) + '\n')

In [ ]:
random.seed(10)
initialize_thread_pool(Value('d', 0), 1)

In [ ]:
random.seed(10)
random.randint(0, 2**64)

In [ ]:
random.randint(0, 2**64)

In [ ]:
random.seed(10)

initialize_pool(Value('d', 0), 1)
model_fitter = ModelFitter(DefaultModel(), 
                           random_sample = random_sample, 
                           verbose = verbose, 
                           threads = threads)

params, loglik_train, loglik_test = model_fitter.cross_validate(fold_data, fold_number - 1)
with (output_path / ("params" + str(i + 1) + ".csv")).open('w') as f:
    f.write(','.join(str(x) for x in params))
with (output_path / ("lltrain" + str(i + 1) + ".csv")).open('w') as f:
    f.write(','.join(str(x) for x in loglik_train))
with (output_path / ("lltest" + str(i + 1) + ".csv")).open('w') as f:
    f.write(' '.join(str(x) for x in loglik_test) + '\n')

# [1] NLL: 25.4614 Params: [1.053, 0.68, 0.147, 0.292, 0.539, 0.952, -1.899, -1.372, -4.072, -3.799]
# [2] NLL: 21.8845 Params: [1.334, 0.294, 0.414, 0.005, 1.939, -4.663, -4.106, 4.424, -3.281, 2.393]
# [3] NLL: 24.2337 Params: [1.632, 0.782, 0.037, 0.156, 1.082, -3.052, 3.237, 1.782, -0.542, -3.545]
# [4] NLL: 24.5092 Params: [1.913, 0.396, 0.272, 0.423, 1.346, 1.294, 0.444, -4.99, -2.114, 0.264]
# [5] NLL: 24.6297 Params: [2.396, 0.624, 0.304, 0.122, 1.769, -2.373, 1.753, -1.235, 2.314, -0.088]
# [6] NLL: 23.613 Params: [2.677, 0.127, 0.006, 0.331, 0.755, 3.271, 4.585, 3.037, 0.02, 3.682]
# [7] NLL: 17.8836 Params: [2.958, 0.95, 0.381, 0.446, 1.61, 4.17, -3.071, 0.669, 3.696, -2.256]
# [8] NLL: 26.7262 Params: [3.239, 0.452, 0.176, 0.225, 0.96, -0.225, -0.903, -2.627, 3.96, 3.975]
# [9] NLL: 26.1641 Params: [3.375, 0.326, 0.09, 0.276, 1.549, -3.745, -0.22, -3.428, 4.937, -2.988]
# [10] NLL: 21.9883 Params: [3.656, 0.824, 0.354, 0.055, 0.887, 1.909, -3.599, 0.22, 2.72, 0.645]